<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Train Practice

## *Data Science Unit 4 Sprint 2 Assignment 4*

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class. Apply regularization techniques to your model. 

*Don't forgot to switch to GPU on Colab!*

## Regularization

Using your best performing model from the previous module, apply each of the following regularization strategies: 
* Early Stopping
* Dropout
* Weight Decay
* Weight Constraint


In [21]:
import pandas as pd
import numpy as np
import tensorflow as ts
import seaborn as sns

In [2]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# required for compatibility bewteen sklearn and keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [6]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.layers import ReLU
import tensorflow as tf
import os


In [3]:
path="/Users/jenniferquigley/DS22/DS-Unit-4-Sprint-2-Neural-Networks/module2-Train/quickdraw10.npz"

In [4]:
def load_quickdraw10(path):
    # Load dataset
    data = np.load(path)
    
    # Split into X & y
    X = data['arr_0']
    y = data['arr_1']
    
    # normalize X
    X = X.astype('float')/255
    
    # Randomly shuffle X & y
    X, y = shuffle(X, y)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)
    
    # Split into train, test sets
    return X_train, y_train, X_test, y_test

In [19]:
X_train, y_train, X_test, y_test = load_quickdraw10(path)

In [37]:
def create_model(units=32):

    # instaniate a Sequential object
    model = Sequential()
    
    # add hidden layer 
    model.add(Dense(units, input_dim=784, activation="relu"))
    
    # add output layer 
    # for multi-class classification, you necessarily must use the softmax act func 
    # number of units = number of unique labels to predict
    model.add(Dense(10, activation = "softmax"))
    
    # complie the model 
    model.compile(loss="sparse_categorical_crossentropy", 
                  optimizer="adam",
                  metrics=["accuracy"])
    
    # return the model 
    return model

In [14]:
model = KerasClassifier(build_fn=create_model)

## Early Stopping

In [17]:
logdir = os.path.join("logs", 
                      "EarlyStopping-Loss")

# instantiate a tensorboard callback object
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

# instantiate a early stopping clallback object 
stop = EarlyStopping(monitor="val_loss", # track the progress of the test loss 
                     min_delta=0.001, # define threshold for what constitutes an improvment 
                     patience=5) # if an improvement doesn't happen after 5 epoches, stop training 

In [18]:
# fit model 
model.fit(X_train, y_train, 
          epochs=15,
          batch_size = 128,
          validation_data=(X_test, y_test), 
          callbacks=[tensorboard_callback, stop])

Epoch 1/15
524/524 [==============================] - 2s 3ms/step - loss: 1.0495 - accuracy: 0.6774 - val_loss: 0.6713 - val_accuracy: 0.8051
Epoch 2/15
524/524 [==============================] - 1s 2ms/step - loss: 0.6073 - accuracy: 0.8235 - val_loss: 0.5816 - val_accuracy: 0.8284
Epoch 3/15
524/524 [==============================] - 1s 2ms/step - loss: 0.5145 - accuracy: 0.8499 - val_loss: 0.5432 - val_accuracy: 0.8407
Epoch 4/15
524/524 [==============================] - 1s 2ms/step - loss: 0.4715 - accuracy: 0.8622 - val_loss: 0.5227 - val_accuracy: 0.8469
Epoch 5/15
524/524 [==============================] - 1s 2ms/step - loss: 0.4315 - accuracy: 0.8737 - val_loss: 0.5026 - val_accuracy: 0.8536
Epoch 6/15
524/524 [==============================] - 1s 2ms/step - loss: 0.4023 - accuracy: 0.8818 - val_loss: 0.4953 - val_accuracy: 0.8568
Epoch 7/15
524/524 [==============================] - 1s 2ms/step - loss: 0.3754 - accuracy: 0.8890 - val_loss: 0.4920 - val_accuracy: 0.8585
Epoch 

## Weight Decay
- shrinks down the weights

In [52]:
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import tensorflow as tf
import os

In [53]:
def create_model(units=32):

    # instaniate a Sequential object
    model = Sequential()
    
    # add hidden layer 
    model.add(Dense(units, input_dim=784, activation="relu"))
    
    # add output layer 
    # for multi-class classification, you necessarily must use the softmax act func 
    # number of units = number of unique labels to predict
    model.add(Dense(10, activation = "softmax", kernel_regularizer=regularizers.l2(0.01)))
    
    # complie the model 
    model.compile(loss="sparse_categorical_crossentropy", 
                  optimizer="adam",
                  metrics=["accuracy"])
    
    # return the model 
    return model

In [40]:
model = KerasClassifier(build_fn=create_model)

logdir = os.path.join("logs", 
                      "EarlyStopping+L2_WeightDecay")

tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(X_train, y_train, 
          epochs=100, 
          validation_data=(X_test, y_test), 
          callbacks=[tensorboard_callback, stop])

Epoch 1/100
2094/2094 [==============================] - 3s 1ms/step - loss: 1.1819 - accuracy: 0.6769 - val_loss: 0.8061 - val_accuracy: 0.7875
Epoch 2/100
2094/2094 [==============================] - 3s 1ms/step - loss: 0.7752 - accuracy: 0.7977 - val_loss: 0.7130 - val_accuracy: 0.8111
Epoch 3/100
2094/2094 [==============================] - 3s 1ms/step - loss: 0.6992 - accuracy: 0.8164 - val_loss: 0.6734 - val_accuracy: 0.8205
Epoch 4/100
2094/2094 [==============================] - 3s 1ms/step - loss: 0.6516 - accuracy: 0.8277 - val_loss: 0.6464 - val_accuracy: 0.8259
Epoch 5/100
2094/2094 [==============================] - 3s 1ms/step - loss: 0.6227 - accuracy: 0.8363 - val_loss: 0.6250 - val_accuracy: 0.8308
Epoch 6/100
2094/2094 [==============================] - 3s 1ms/step - loss: 0.5959 - accuracy: 0.8418 - val_loss: 0.6141 - val_accuracy: 0.8338
Epoch 7/100
2094/2094 [==============================] - 3s 1ms/step - loss: 0.5732 - accuracy: 0.8473 - val_loss: 0.6036 - val_ac

In [41]:
model.summary()

AttributeError: 'KerasClassifier' object has no attribute 'summary'

In [42]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [43]:
%tensorboard --logdir=logs/ --host localhost --port 8090

## Weight Constraint

In [48]:
from tensorflow.keras.constraints import MaxNorm

In [49]:
def create_model(units=32):

    # instaniate a Sequential object
    model = Sequential()
    
    wc = MaxNorm(max_value=2)
    
    # add hidden layer 
    model.add(Dense(units, input_dim=784, activation="relu"))
    
    # add output layer 
    # for multi-class classification, you necessarily must use the softmax act func 
    # number of units = number of unique labels to predict
    model.add(Dense(10, activation = "softmax", kernel_regularizer=wc))
    
    # complie the model 
    model.compile(loss="sparse_categorical_crossentropy", 
                  optimizer="adam",
                  metrics=["accuracy"])
    
    # return the model 
    return model

In [51]:
model = KerasClassifier(build_fn=create_model)

logdir = os.path.join("logs", 
                      "EarlyStopping+L1_WeightConstraint")

tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(X_train, y_train, 
          epochs=100, 
          validation_data=(X_test, y_test), 
          callbacks=[tensorboard_callback, stop])

Epoch 1/100


ValueError: in user code:

    /Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-2-Neural-Networks-f6C6DwwC/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-2-Neural-Networks-f6C6DwwC/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-2-Neural-Networks-f6C6DwwC/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-2-Neural-Networks-f6C6DwwC/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return sel

## Deploy

Save your model's weights using the Checkpoint function. Try reloading the model and making inference on your validation dataset.

In [55]:
model = create_model()
model.fit(X_train,y_train, epochs=5)


!mkdir -p saved_model
model.save('saved_model/my_model') 

Epoch 1/5
2094/2094 [==============================] - 2s 1ms/step - loss: 1.1565 - accuracy: 0.6778
Epoch 2/5
2094/2094 [==============================] - 2s 1ms/step - loss: 0.7755 - accuracy: 0.7938
Epoch 3/5
2094/2094 [==============================] - 2s 1ms/step - loss: 0.6943 - accuracy: 0.8160
Epoch 4/5
2094/2094 [==============================] - 2s 1ms/step - loss: 0.6521 - accuracy: 0.8265
Epoch 5/5
2094/2094 [==============================] - 2s 973us/step - loss: 0.6255 - accuracy: 0.8306
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [56]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 32)                25120     
_________________________________________________________________
dense_19 (Dense)             (None, 10)                330       
Total params: 25,450
Trainable params: 25,450
Non-trainable params: 0
_________________________________________________________________


In [57]:
new_model.evaluate(X_test, y_test)

1032/1032 [==============================] - 1s 689us/step - loss: 0.6368 - accuracy: 0.8282


[0.636752724647522, 0.828151524066925]

### Stretch Goals
- Mount your Google Drive to Colab to persist your model checkpoint files. 
- Research L2 normalization (weight decay)
- Write a custom callback function to stop training after you reach .88 validation accuracy. 
- Select a new dataset and apply a neural network to it.
- Research TensorFlow Serving
- Play [QuickDraw](https://quickdraw.withgoogle.com/data)
- Create a static webpage using TensorFlow.js to serve a model. Check out [Teachable Machine Learning](https://teachablemachine.withgoogle.com/) for ideas. 